In [1]:
import gzip
import os 
import json
import re

import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

from dotenv import load_dotenv

from utils import config
from utils import utilities

In [2]:
conn = psycopg2.connect(
            dbname=config.DATABASE['DBNAME'],
            host=config.DATABASE['HOST'],
            user=config.DATABASE['USER'],
            password=config.DATABASE['PASSWORD'],
            port=config.DATABASE['PORT'],
        )

engine = create_engine(
            f"postgresql://{config.DATABASE['USER']}:"
            f"{config.DATABASE['PASSWORD']}@"
            f"{config.DATABASE['HOST']}:"
            f"{config.DATABASE['PORT']}/"
            f"{config.DATABASE['DBNAME']}"
        )

In [4]:
"""
all_ids = []
with gzip.open(config.TWEETS_PATH, 'rb') as f:
    for row_number, current_tweet in enumerate(f):
        all_ids.append(json.loads(current_tweet.decode(encoding='utf-8'))['id'])

print(f'Unique id amount: {len(set(all_ids))}, out of total id amount: {len(all_ids)}')
"""

"\nall_ids = []\nwith gzip.open(config.TWEETS_PATH, 'rb') as f:\n    for row_number, current_tweet in enumerate(f):\n        all_ids.append(json.loads(current_tweet.decode(encoding='utf-8'))['id'])\n\nprint(f'Unique id amount: {len(set(all_ids))}, out of total id amount: {len(all_ids)}')\n"

In [5]:
batch_size = 100000

data_rows = []
conversations_existing_ids = utilities.run_written_query('SELECT id FROM conversations', to_dataframe=True, option='from_string').id.astype(str).values
with gzip.open(config.TWEETS_PATH, 'rb') as f:
    for row_number, current_tweet in enumerate(f):
        data_rows.append(json.loads(current_tweet.decode(encoding='utf-8')))
        if (row_number+1) % batch_size == 0:
            print(row_number+1)
            break

/home/xschon/Desktop/PDT/Z1/utils/utilities.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, con=conn)


100000


In [6]:
tweets = pd.DataFrame(data_rows)
metrics = pd.DataFrame(tweets.public_metrics.to_list())
tweets[metrics.columns] = metrics
tweets.rename(columns={'text' : 'content',
                       'lang' : 'language',

                    }, inplace=True)

tweets.columns

Index(['author_id', 'conversation_id', 'created_at', 'entities', 'id',
       'language', 'possibly_sensitive', 'public_metrics', 'referenced_tweets',
       'reply_settings', 'source', 'content', 'context_annotations',
       'attachments', 'in_reply_to_user_id', 'geo', 'withheld', 'like_count',
       'quote_count', 'reply_count', 'retweet_count'],
      dtype='object')

In [7]:
# TODO authors uploading when uploading conversations_table
# TODO drop duplicates of ids

conversations_table = tweets[['id', 'author_id', 'content', 'possibly_sensitive', 'language',
        'source', 'retweet_count', 'reply_count', 'like_count', 'quote_count', 'created_at']]

In [198]:
# Find all referenced tweets - iterate over existing array of references for each tweet
refs = tweets.dropna(subset='referenced_tweets')[['id', 'referenced_tweets']]
conversation_references_table = pd.DataFrame(columns=['conversation_id', 'parent_id', 'type'])

for layer in range(refs.referenced_tweets.apply(lambda x : len(x)).max()):
    # Select all references from given layer and find tweets they reffer to
    layer_references = refs.referenced_tweets.apply(lambda x : x[layer] if(len(x) > layer) else None).dropna()
    conv_refs = pd.DataFrame(layer_references.to_list(), index=layer_references.index)
    conv_refs.rename(columns={'id' : 'parent_id'}, inplace=True)
    conv_refs = (conv_refs.join(refs.id).rename(columns={'id' : 'conversation_id'}))

    conversation_references_table = pd.concat([conversation_references_table, conv_refs])

74074
17


In [302]:
links_table = pd.DataFrame(columns=['conversation_id', 'url', 'title', 'description'])
links_raw = pd.DataFrame(tweets.dropna(subset='entities').entities.to_list()).dropna(subset='urls').urls


for layer in range(links_raw.apply(lambda x : len(x)).max()):
    layer_links = links_raw.apply(lambda x : x[layer] if (len(x) > layer) else None).dropna()
    tmp_links = pd.DataFrame(layer_links.to_list(), index=layer_links.index)
    tmp_links = tmp_links.join(tweets.id).rename(columns={'id' : 'conversation_id'})

    links_table = pd.concat([links_table, tmp_links])[list(links_table.columns)]

In [339]:
annotations_table.dtypes

conversation_id     object
value               object
type                object
probability        float64
dtype: object

In [343]:
tmp_annotations

,end,normalized_text,probability,start,type,conversation_id
2,22,Joe Biden,0.9976,14,Person,1496732916281491458
3,72,CHABELO,0.5075,66,Person,1496732914503069698
5,29,Ukraine,0.8356,23,Place,1496732916784582659
13,48,Latinoamérica,0.8778,36,Place,1496732918009548804
14,44,Ukraine,0.8824,38,Place,1496732920286875649
...,...,...,...,...,...,...
99500,36,Biden,0.9821,32,Person,1496765509638889472
99505,42,,0.4662,42,Person,1496765508150046728
99513,57,Nobel Peace Prize,0.3290,41,Other,1496765511257968642
99514,121,Tercera Guerra Mundial,0.8203,100,Other,1496765510913957890


In [348]:
annotations_table = pd.DataFrame(columns=['conversation_id', 'value', 'type', 'probability'])
annotations_raw = links_raw = pd.DataFrame(tweets.dropna(subset='entities').entities.to_list()).dropna(subset='annotations').annotations

for layer in range(annotations_raw.apply(lambda x : len(x)).max()):
    layer_annotations = annotations_raw.apply(lambda x : x[layer] if (len(x)>layer) else None).dropna()
    tmp_annotations  = pd.DataFrame(layer_annotations.to_list(), index=layer_annotations.index)
    tmp_annotations.rename(columns={'normalized_text' : 'values'}, inplace=True)
    tmp_annotations = tmp_annotations.join(tweets.id).rename(columns={'id' : 'conversation_id'})
    
    annotations_table = pd.concat([annotations_table, tmp_annotations])[list(annotations_table.columns)]